In [14]:
import numpy as np 


In [ ]:

mma_m, mma_n, mma_k = 16,8,16 

#okay think about this, if we have acc_per_warp_m, acc_per_warp_n 
#then this (always mma tile granular) induces a acc_per_warp_m x K by K x acc_per_warp_n matmul. 

#this is actually a shared memory situation so it s actually a 

# acc_per_warp_m x BK by BK x acc_per_warp_n matmul. 

#indeed, once chooses a paramater BK_stages 
#so the done matmul by 1 warp is of shape (mma_m*acc_per_warp_m, mma_n*acc_per_warp_n, mma_k*BK)
#done in stages

#the matmul happens, for each loop its is (in tile granular, acc_per_warp_m, acc_per_warp_n, BK_stages)
# 

acc_per_warp_m = 2
acc_per_warp_n = 4
BK_stages = 2 

num_c_acc = acc_per_warp_m*acc_per_warp_n 
num_a_frags = acc_per_warp_m*BK_stages 
num_b_frags = acc_per_warp_n*BK_stages 

num_32_bit_regs = ((num_c_acc*4) + ((num_a_frags*2)+(num_b_frags)))*32

In [7]:
num_32_bit_regs

48

In [ ]:
"""

shared As[2],Bs[2]

bar empty[2] 
bar full[2] 

#prologue

waste_token = (empty[0].arrive())
waste_token = (empty[1].arrive())

if (threadIdx.x == 0):

  empty[0].wait(empty[0].arrive())
  
  TMA_fill(As[0], Bs[0], full[0] k_iter = 0)
  waste_token = barrier_arrive_tx(full[0], As_bytes + Bs_bytes) 
  
else: 
  waste_token = full[0].arrive()
  
  
for bk in [0,bk_iters): 
  stage = (bk % 2)
  next = (bk + 1) % 2 
  
  if (threadIdx == 0 and bk + 1 < bk_iters): 
    empty[next].wait(empty[next].arrive())
    TMA_fill(As[next],Bs[next],full[next], k_iter = bk + 1)
    waste_token = barrier_arrive_tx(full[next], AS_bytes + Bs_bytes)
  else: 
    waste_token = full[next].arrive()
  
  
  full[stage].wait(full[stage].arrive())    
  COMPUTE(As[stage], Bs[stage]) 
  
   waste_token = empty[stage].arrive()


"""